In [1]:
import gc
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
#import lightgbm as lgb

In [2]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [3]:
import pandas as pd
import numpy as np

In [4]:
art = pd.read_csv("articles.csv")
art.shape

(105542, 25)

In [5]:
art.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [6]:
cst = pd.read_csv("customers.csv")
cst.shape

(1371980, 7)

In [7]:
cst.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [8]:
#trns = pd.read_csv("transactions_train.csv", nrows=999999)
trns = pd.read_csv("transactions_train.csv")
trns.shape

(31788324, 5)

In [9]:
trns.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


## Collab Filtering

similarity between customers is used for recommendation. we find customer based their purchasing patterns. similarity is calculated using 

### Read CSV-short

In [11]:
art_cst_merge=pd.read_csv ('articles_for_merge.csv')
art_cst_merge.shape

(383453, 9)

In [12]:
art_cst_merge.head()

,customer_id,article_id,price,t_dat,prod_name,product_type_name,product_group_name,index_name,perceived_colour_value_name
0,001ddeb8fb74fec5693116da83b488e05ee9a9e179f3fd...,712587003,0.020254,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
1,a80847aee5b274e644a4f0059e723c11f9410ef5a3da8e...,712587003,0.010153,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
2,abe2f85c340f91805173b08461c5d4e4fd502cff0960e0...,712587003,0.022017,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
3,abe2f85c340f91805173b08461c5d4e4fd502cff0960e0...,712587003,0.022017,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
4,0fd4835666153d82838b3bccd0e9e73066e26329e45517...,712587003,0.019814,2020-01-03,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark


In [13]:
pd.DataFrame(art_cst_merge.groupby(['product_group_name'])['article_id'].nunique())

,article_id
product_group_name,
Garment Full body,332
Garment Lower body,309
Garment Upper body,639


In [14]:
cst_art_buy = art_cst_merge.groupby(["customer_id", "article_id"])["t_dat"].count().reset_index()
cst_art_buy.columns = ["customer_id", "article_id", "count"]

In [15]:
cst_art_buy.head()

,customer_id,article_id,count
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,2
1,000114c6996ef5703a8d455faf2103f8488d3928348e07...,733441003,1
2,00012315fd38859ff2c446876ca507abbcbcf582d0e266...,690936001,1
3,00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d...,842605015,1
4,0001ab2ebc1bb9a21d135e2fefdb11f12bee5c74ab2984...,772773002,1


In [16]:
# create flag for an itme purchased by customer
cst_art_buy['IsPurchased'] = 1

In [17]:
uniqueCst = list(cst_art_buy.customer_id.unique())
uniqueArt = list(cst_art_buy.article_id.unique()) 

In [18]:
print('Total Customers = '+str(f'{len(uniqueCst):,}'))
print('total articles/products = '+str(f'{len(uniqueArt):,}'))
print('Total Transactions = '+str(f'{len(art_cst_merge):,}'))

Total Customers = 209,267
total articles/products = 1,280
Total Transactions = 383,453


In [19]:
#cst_art_buy_short = cst_art_buy[cst_art_buy['article_id'].isin()]

In [20]:
# make Cst-Art Matrix
cst_art_matrix = pd.pivot_table(cst_art_buy, values='IsPurchased', index='customer_id', columns='article_id')

In [21]:
cst_art_matrix.head()

article_id,186372045,220094015,294008050,316441024,356174004,399061018,399061023,399256005,400296010,445419010,...,931769003,932024001,933111001,934122004,934211005,934296001,935548001,938622001,943097001,946748004
customer_id,,,,,,,,,,,,,,,,,,,,,
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000114c6996ef5703a8d455faf2103f8488d3928348e0739c92ff9e8897932db,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00012315fd38859ff2c446876ca507abbcbcf582d0e266b1b696941c16e777a2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d60c6d7be26f06264d6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001ab2ebc1bb9a21d135e2fefdb11f12bee5c74ab29842a6d2ffd71316bf048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
cst_art_matrix_reset = cst_art_matrix.reset_index().rename_axis(None, axis=1) 
cst_art_matrix_reset.head() 

,customer_id,186372045,220094015,294008050,316441024,356174004,399061018,399061023,399256005,400296010,...,931769003,932024001,933111001,934122004,934211005,934296001,935548001,938622001,943097001,946748004
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000114c6996ef5703a8d455faf2103f8488d3928348e07...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00012315fd38859ff2c446876ca507abbcbcf582d0e266...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001ab2ebc1bb9a21d135e2fefdb11f12bee5c74ab2984...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# fill NaN with 0
cst_art_matrix_reset=cst_art_matrix_reset.fillna(0)

In [24]:
cst_art_matrix_reset.head()

,customer_id,186372045,220094015,294008050,316441024,356174004,399061018,399061023,399256005,400296010,...,931769003,932024001,933111001,934122004,934211005,934296001,935548001,938622001,943097001,946748004
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000114c6996ef5703a8d455faf2103f8488d3928348e07...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00012315fd38859ff2c446876ca507abbcbcf582d0e266...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0001ab2ebc1bb9a21d135e2fefdb11f12bee5c74ab2984...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


we have customer-item matrix

In [25]:
# data with articles matrix only
cst_art_matrix_reset_art = cst_art_matrix_reset.drop('customer_id',1)

In [26]:
# normalize data
cst_art_matrix_reset_art = cst_art_matrix_reset_art/np.sqrt(np.square(cst_art_matrix_reset_art).sum(axis=0))  


In [27]:
cst_art_matrix_reset_art.head()

,186372045,220094015,294008050,316441024,356174004,399061018,399061023,399256005,400296010,445419010,...,931769003,932024001,933111001,934122004,934211005,934296001,935548001,938622001,943097001,946748004
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# Using .dot Vectors to get Cosine Similarity
cst_art_matrix_reset_art_dot_vector = cst_art_matrix_reset_art.transpose().dot(cst_art_matrix_reset_art)
cst_art_matrix_reset_art_dot_vector.head()

,186372045,220094015,294008050,316441024,356174004,399061018,399061023,399256005,400296010,445419010,...,931769003,932024001,933111001,934122004,934211005,934296001,935548001,938622001,943097001,946748004
186372045,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
220094015,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
294008050,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316441024,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.001847,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
356174004,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


we have item-item similarity matrix

In [101]:

k = 5 # number of items to recommend
cst_art_neigbour = pd.DataFrame(index=cst_art_matrix_reset_art_dot_vector.columns,columns=range(1,k+2))

# fill in recommended item names
for i in range(0,len(cst_art_matrix_reset_art_dot_vector.columns)): 
    cst_art_neigbour.iloc[i,:k+1] = cst_art_matrix_reset_art_dot_vector.iloc[0:,i].sort_values(ascending=False)[:k+1].index 
 


find closest neighbour for each item based on similarity value

In [102]:
#cst_art_neigbour.to_excel('Export_art_art_neighbours.xlsx')

In [103]:
cst_art_neigbour.head()

,1,2,3,4,5,6
186372045,186372045,562251001,489838010,563635003,619881001,523642006
220094015,220094015,780729004,714472001,716348008,753724002,784229002
294008050,294008050,751941002,867023003,771602001,820037001,822199006
316441024,316441024,575542003,703991003,808305003,806516001,523642006
356174004,356174004,186372045,820671001,822237001,822199006,822180001


In [104]:
cst_art_similarity = pd.DataFrame(index=cst_art_matrix_reset.index,columns=cst_art_matrix_reset.columns) 
cst_art_similarity.iloc[:,:1] = cst_art_matrix_reset.iloc[:,:1] 
cst_art_similarity.head()

,customer_id,186372045,220094015,294008050,316441024,356174004,399061018,399061023,399256005,400296010,...,931769003,932024001,933111001,934122004,934211005,934296001,935548001,938622001,943097001,946748004
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000114c6996ef5703a8d455faf2103f8488d3928348e07...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00012315fd38859ff2c446876ca507abbcbcf582d0e266...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001ab2ebc1bb9a21d135e2fefdb11f12bee5c74ab2984...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Scikit Short

In [29]:
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

In [30]:
def art_art_similarity(cst_ids,art_ids):
    art_cst_matrix = csr_matrix(([1]*len(cst_ids), (art_ids, cst_ids)))
    similarity = cosine_similarity(art_cst_matrix)
    return similarity, art_cst_matrix

In [31]:
def get_recommendations_from_similarity(similarity_matrix, art_cst_matrix, top_n=3):
    cst_art_matrix = csr_matrix(art_cst_matrix.T)
    cst_art_score = cst_art_matrix.dot(similarity_matrix) # sum of similarities to all purchased products
    recommend_for_cust = []
    for user_id in range(cst_art_score.shape[0]):
        scores = cst_art_score[user_id, :]
        purchased_art = cst_art_matrix.indices[cst_art_matrix.indptr[user_id]:cst_art_matrix.indptr[user_id+1]]
        scores[purchased_art] = -1 # not recommend purchased articles
        top_products_ids = np.argsort(scores)[-top_n:][::-1]
        recommendations = pd.DataFrame(
            top_products_ids.reshape(1, -1),
            index=[user_id],
            columns=['Top%s' % (i+1) for i in range(top_n)])
        recommend_for_cust.append(recommendations)
    return pd.concat(recommend_for_cust)

In [32]:
def get_recommendations(purchase_data):
    cst_label_encoder = LabelEncoder()
    cst_ids = cst_label_encoder.fit_transform(purchase_data.customer_id)
    art_label_encoder = LabelEncoder()
    art_ids = art_label_encoder.fit_transform(purchase_data.article_id)
    # compute recommendations
    similarity, art_cst_matrix = art_art_similarity(cst_ids, art_ids)
    recommendations = get_recommendations_from_similarity(similarity, art_cst_matrix)
    recommendations.index = cst_label_encoder.inverse_transform(recommendations.index)
    for i in range(recommendations.shape[1]):
        recommendations.iloc[:, i] = art_label_encoder.inverse_transform(recommendations.iloc[:, i])
    return recommendations

In [33]:
purchase_data = cst_art_buy.drop('count',1)

In [34]:
purchase_data.shape

(329158, 3)

In [35]:
purchase_data.customer_id.nunique()

209267

In [36]:
purchase_data.article_id.nunique()

1280

In [37]:
purchase_data.IsPurchased.value_counts()

1    329158
Name: IsPurchased, dtype: int64

In [38]:
recommendations = get_recommendations(purchase_data)

KeyboardInterrupt: 

In [131]:
recommendations.head()

,Top1,Top2,Top3
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,625358003,750424010,690936023
000114c6996ef5703a8d455faf2103f8488d3928348e0739c92ff9e8897932db,608570002,691834001,832253002
00012315fd38859ff2c446876ca507abbcbcf582d0e266b1b696941c16e777a2,690936014,690936023,690936021
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d60c6d7be26f06264d6,721636001,751471001,675070015
0001ab2ebc1bb9a21d135e2fefdb11f12bee5c74ab29842a6d2ffd71316bf048,778064003,767032002,610776001


In [132]:
recommendations.shape

(209267, 3)

### ALS Short

In [34]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

In [35]:
art_cst_merge=pd.read_csv ('articles_for_merge.csv', dtype={'article_id': str}, parse_dates=['t_dat'])
art_cst_merge.shape

(383453, 9)

In [36]:
art_cst_merge.head()

,customer_id,article_id,price,t_dat,prod_name,product_type_name,product_group_name,index_name,perceived_colour_value_name
0,001ddeb8fb74fec5693116da83b488e05ee9a9e179f3fd...,712587003,0.020254,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
1,a80847aee5b274e644a4f0059e723c11f9410ef5a3da8e...,712587003,0.010153,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
2,abe2f85c340f91805173b08461c5d4e4fd502cff0960e0...,712587003,0.022017,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
3,abe2f85c340f91805173b08461c5d4e4fd502cff0960e0...,712587003,0.022017,2020-01-02,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark
4,0fd4835666153d82838b3bccd0e9e73066e26329e45517...,712587003,0.019814,2020-01-03,Bow HW paperwaist,Trousers,Garment Lower body,Ladieswear,Dark


In [37]:
art_cst_merge= art_cst_merge[['customer_id','article_id','price','t_dat']]

In [38]:
art_cst_merge['t_dat'].max()

Timestamp('2020-09-22 00:00:00')

In [39]:
art_cst_merge['t_dat'].min()

Timestamp('2020-01-02 00:00:00')

Q3 july, august, sept
train : July 1st, 2020 to Sept 15th,2020
test: Sept 16, 2020 to Sept 22, 2020 (last 7 days in data)


#baseline: get most popular items in last 15 days i.e. from  say Sept 1st,2020 to Sept 15th,2020
#soi: stock keep changing so reccos depends on stock too + we are not reccomending based on item and customer data so recent transactions are better



2020-09-23 to 2020-09-29 is kaggles test period
2020-09-16 to 2020-09-29 is my test period
get most popular items in last 2 weeks i.e. from  say 2020-09-01 to 2020-09-15

2020-09-01 to 2020-09-22 is my test period
get most popular items in last 1 month i.e. from  say 2020-08-01 to 2020-08-31

In [40]:
art_cst_train = art_cst_merge[('2020-06-30' < art_cst_merge['t_dat'])  & (art_cst_merge['t_dat'] < '2020-09-16')]
art_cst_train.shape

(114366, 4)

In [41]:
#art_cst_train['t_dat'].max()
#art_cst_train['t_dat'].min()

In [42]:
art_cst_test = art_cst_merge[art_cst_merge['t_dat'] > '2020-09-15']
art_cst_test.shape

(9115, 4)

In [43]:
#art_cst_test['t_dat'].max()
#art_cst_test['t_dat'].min()

Training

In [44]:
ALL_USERS = art_cst_train['customer_id'].unique().tolist()  #list of unique customers
ALL_ITEMS = art_cst_train['article_id'].unique().tolist()   #list of unique items

In [45]:
len(ALL_USERS)

77142

In [46]:
len(ALL_ITEMS)

915

In [47]:
user_ids = dict(list(enumerate(ALL_USERS))) #list of unique customers in dictionary format 
item_ids = dict(list(enumerate(ALL_ITEMS)))  #list of unique items in dictionary format

In [48]:
user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

In [49]:
art_cst_train['user_id'] = art_cst_train['customer_id'].map(user_map)
art_cst_train['item_id'] = art_cst_train['article_id'].map(item_map)

<ipython-input-49-d877e3d8f3df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_cst_train['user_id'] = art_cst_train['customer_id'].map(user_map)
<ipython-input-49-d877e3d8f3df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_cst_train['item_id'] = art_cst_train['article_id'].map(item_map)


Create coo_matrix (user x item) and csr matrix (user x item)

In [50]:
row = art_cst_train['user_id'].values
col = art_cst_train['item_id'].values
data = np.ones(art_cst_train.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

<77142x915 sparse matrix of type '<class 'numpy.float64'>'
	with 114366 stored elements in COOrdinate format>

Run ALS with coo_train

In [51]:
%%time
model = implicit.als.AlternatingLeastSquares(factors=20, iterations=2)
model.fit(coo_train)

  0%|          | 0/2 [00:00<?, ?it/s]

Wall time: 299 ms


testing ALS

In [52]:
ALL_USERS = art_cst_test['customer_id'].unique().tolist()  #list of unique customers
ALL_ITEMS = art_cst_test['article_id'].unique().tolist()   #list of unique items

In [53]:
len(ALL_USERS)

7048

In [54]:
len(ALL_ITEMS)

475

In [55]:
user_ids = dict(list(enumerate(ALL_USERS))) #list of unique customers in dictionary format 
item_ids = dict(list(enumerate(ALL_ITEMS)))  #list of unique items in dictionary format

In [56]:
user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

In [57]:
art_cst_test['user_id'] = art_cst_test['customer_id'].map(user_map)
art_cst_test['item_id'] = art_cst_test['article_id'].map(item_map)

<ipython-input-57-81ae858df0bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_cst_test['user_id'] = art_cst_test['customer_id'].map(user_map)
<ipython-input-57-81ae858df0bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_cst_test['item_id'] = art_cst_test['article_id'].map(item_map)


Create coo_matrix (user x item) and csr matrix (user x item)

In [58]:
row = art_cst_test['user_id'].values
col = art_cst_test['item_id'].values
data = np.ones(art_cst_test.shape[0])
coo_test = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_test

<7048x475 sparse matrix of type '<class 'numpy.float64'>'
	with 9115 stored elements in COOrdinate format>

In [59]:
csr_train = coo_train.tocsr()
csr_test = coo_test.tocsr()

In [60]:
map12 = mean_average_precision_at_k(model, csr_train, csr_test, K=12, num_threads=4)
#print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")

  0%|          | 0/7048 [00:00<?, ?it/s]

In [61]:
map12

0.0016252969058729552

In [62]:
from implicit.evaluation import ndcg_at_k

In [63]:
ndcg_at_k(model, csr_train, csr_test, K=15, show_progress=True,num_threads=4)

  0%|          | 0/7048 [00:00<?, ?it/s]

0.002876196636172954

In [64]:
from implicit.evaluation import precision_at_k

precision_at_k(model,csr_train, csr_test, K=12, show_progress=True,num_threads=4)

  0%|          | 0/7048 [00:00<?, ?it/s]

0.006186587478346944